## *Import Libraries*

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

## *Load Cleaned Data*

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
%time df = pd.read_csv('clean_final.csv') # parse_dates = ['FL_DATE'])

Wall time: 24.8 s


In [5]:
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,AIR_TIME,DISTANCE,DIV_AIRPORT_LANDINGS
0,2019,1,1,3,4,2019-01-03,9E,20363,9E,N195PQ,5121,15412,35412,TYS,"Knoxville, TN",TN,47,Tennessee,54,10397,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,1140,1205.0,25.0,25.0,1.0,1.0,1100-1159,30.0,1311.0,4.0,1250,1.0,1.0,1200-1259,0.0,0.0,70.0,36.0,152.0,0
1,2019,1,1,4,5,2019-01-04,9E,20363,9E,N919XJ,5121,15412,35412,TYS,"Knoxville, TN",TN,47,Tennessee,54,10397,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,1140,1250.0,70.0,70.0,1.0,4.0,1100-1159,35.0,1403.0,9.0,1250,1.0,5.0,1200-1259,0.0,0.0,70.0,38.0,152.0,0
2,2019,1,1,5,6,2019-01-05,9E,20363,9E,N316PQ,5122,10397,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14783,34783,SGF,"Springfield, MO",MO,29,Missouri,64,950,956.0,6.0,6.0,0.0,0.0,0900-0959,20.0,1040.0,3.0,1051,0.0,-1.0,1000-1059,0.0,0.0,121.0,84.0,563.0,0
3,2019,1,1,6,7,2019-01-06,9E,20363,9E,N325PQ,5122,10397,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14783,34783,SGF,"Springfield, MO",MO,29,Missouri,64,950,945.0,-5.0,0.0,0.0,-1.0,0900-0959,16.0,1026.0,3.0,1053,0.0,-2.0,1000-1059,0.0,0.0,123.0,85.0,563.0,0
4,2019,1,1,7,1,2019-01-07,9E,20363,9E,N904XJ,5122,10397,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14783,34783,SGF,"Springfield, MO",MO,29,Missouri,64,950,947.0,-3.0,0.0,0.0,-1.0,0900-0959,25.0,1040.0,4.0,1053,0.0,-1.0,1000-1059,0.0,0.0,123.0,88.0,563.0,0


In [6]:
df.shape

(7421901, 47)

In [7]:
df['ORIGIN_CITY_NAME'] = df['ORIGIN_CITY_NAME'].str.split(',', n=1, expand = True)[0]
df['DEST_CITY_NAME'] = df['DEST_CITY_NAME'].str.split(',', n=1, expand = True)[0]

In [8]:
lst = []
for col1 in df.columns:
    for col2 in df.columns:
        if col1 != col2:
            if df[col1].equals(df[col2]) == True:
                lst.append([col1, col2])
lst

[['OP_UNIQUE_CARRIER', 'OP_CARRIER'], ['OP_CARRIER', 'OP_UNIQUE_CARRIER']]

*For predicting if a flight is delayed we cannot use future information like Arrival Data. So we delete the columns*

In [9]:
# 'WHEELS_OFF', 'ARR_TIME', 'ARR_DELAY', 'ARR_DELAY_NEW'
df.drop(['OP_CARRIER', 'OP_CARRIER_AIRLINE_ID', 
         'ORIGIN_AIRPORT_ID', 'ORIGIN', 'ORIGIN_STATE_ABR', 
         'DEP_TIME_BLK', 'TAXI_OUT', 'WHEELS_ON', 'TAXI_IN',
         'DEST_AIRPORT_ID', 'DEST', 'DEST_STATE_ABR', 
         'ARR_DEL15', 'ARR_DELAY_GROUP', 'ARR_TIME_BLK'], axis = 1, inplace = True)

In [10]:
df.shape

(7421901, 32)

In [11]:
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_CITY_MARKET_ID,ORIGIN_CITY_NAME,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_CITY_MARKET_ID,DEST_CITY_NAME,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,CRS_ARR_TIME,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,AIR_TIME,DISTANCE,DIV_AIRPORT_LANDINGS
0,2019,1,1,3,4,2019-01-03,9E,N195PQ,5121,35412,Knoxville,47,Tennessee,54,30397,Atlanta,13,Georgia,34,1140,1205.0,25.0,25.0,1.0,1.0,1250,0.0,0.0,70.0,36.0,152.0,0
1,2019,1,1,4,5,2019-01-04,9E,N919XJ,5121,35412,Knoxville,47,Tennessee,54,30397,Atlanta,13,Georgia,34,1140,1250.0,70.0,70.0,1.0,4.0,1250,0.0,0.0,70.0,38.0,152.0,0
2,2019,1,1,5,6,2019-01-05,9E,N316PQ,5122,30397,Atlanta,13,Georgia,34,34783,Springfield,29,Missouri,64,950,956.0,6.0,6.0,0.0,0.0,1051,0.0,0.0,121.0,84.0,563.0,0
3,2019,1,1,6,7,2019-01-06,9E,N325PQ,5122,30397,Atlanta,13,Georgia,34,34783,Springfield,29,Missouri,64,950,945.0,-5.0,0.0,0.0,-1.0,1053,0.0,0.0,123.0,85.0,563.0,0
4,2019,1,1,7,1,2019-01-07,9E,N904XJ,5122,30397,Atlanta,13,Georgia,34,34783,Springfield,29,Missouri,64,950,947.0,-3.0,0.0,0.0,-1.0,1053,0.0,0.0,123.0,88.0,563.0,0


In [12]:
df.dropna(inplace = True)

*Extract the hour of day from CRS_DEP_TIME*

In [13]:
df['CRS_DEP_TIME'] = df['CRS_DEP_TIME'].astype(int).astype(str).apply(lambda x: x.zfill(4))

# Convert to number of minutes elapsed since 00:00, from 0 to 1439: (= 24 hours x 60 minutes) 
df['CRS_DEP_TIME_MNT'] = df['CRS_DEP_TIME'].str[:2].astype(int) * 60 + df['CRS_DEP_TIME'].str[2:].astype(int)

# We normalize x values to match with the 0-2π cycle
df['CRS_DEP_TIME_MNT_norm'] = 2 * math.pi * df['CRS_DEP_TIME_MNT'] / df['CRS_DEP_TIME_MNT'].max()

In [14]:
df['DEP_TIME'] = df['DEP_TIME'].astype(int).astype(str).apply(lambda x: x.zfill(4))

# Convert to number of minutes elapsed since 00:00, from 0 to 1439: (= 24 hours x 60 minutes)
df['DEP_TIME_MNT'] = df['DEP_TIME'].str[:2].astype(int) * 60 + df['DEP_TIME'].str[2:].astype(int)

# We normalize x values to match with the 0-2π cycle
df['DEP_TIME_norm'] = 2 * math.pi * df['DEP_TIME_MNT'] / df['DEP_TIME_MNT'].max()

*FLIGHT_COUNT per day*

In [15]:
df = df.sort_values(by = ['FL_DATE', 'CRS_DEP_TIME_MNT_norm']).reset_index(drop = True)

In [16]:
df['FLIGHT_COUNT'] = df.dropna().groupby(['TAIL_NUM', 'FL_DATE']).cumcount()+1
df['FLIGHT_COUNT'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=int64)

*Prior info Origin, Delay, Hour*

In [17]:
def get_prior_info(df, column, name, replace_null):
    
    df = df.sort_values(by = ['TAIL_NUM', 'FL_DATE']).reset_index(drop = True)
    df[name] = df.loc[df['FLIGHT_COUNT'].shift(-1)>1, column]
    df[name] = df[name].shift()
    df[name] = df[name].fillna(value=replace_null)
    return df

In [18]:
df = get_prior_info(df, 'ORIGIN_CITY_NAME', 'PRIOR_ORIGIN_CITY_NAME', replace_null='None')
df = get_prior_info(df, 'DEP_DEL15', 'PRIOR_DELAY', replace_null=0)
df = get_prior_info(df, 'CRS_DEP_TIME_MNT_norm', 'PRIOR_DEP_MNT', replace_null=0)

In [19]:
df[(df['MONTH'] == 1) 
            & (df['DAY_OF_MONTH'] == 3)
            & (df['TAIL_NUM'] == '215NV')].sort_values('DEP_TIME')[['TAIL_NUM',
                                                                    'FL_DATE',
                                                                    'FLIGHT_COUNT',
                                                                    'ORIGIN_CITY_NAME',
                                                                    'PRIOR_ORIGIN_CITY_NAME',
                                                                    'DEP_DEL15',
                                                                    'PRIOR_DELAY',
                                                                    'CRS_DEP_TIME_MNT_norm',
                                                                    'PRIOR_DEP_MNT']]

,TAIL_NUM,FL_DATE,FLIGHT_COUNT,ORIGIN_CITY_NAME,PRIOR_ORIGIN_CITY_NAME,DEP_DEL15,PRIOR_DELAY,CRS_DEP_TIME_MNT_norm,PRIOR_DEP_MNT
7,215NV,2019-01-03,1,Asheville,None,0.0,0.0,2.117682,0.000000
8,215NV,2019-01-03,2,Fort Lauderdale,Asheville,0.0,0.0,2.820666,2.117682
9,215NV,2019-01-03,3,Asheville,Fort Lauderdale,0.0,0.0,3.541114,2.820666
10,215NV,2019-01-03,4,Baltimore,Asheville,0.0,0.0,4.104374,3.541114
11,215NV,2019-01-03,5,Savannah,Baltimore,0.0,0.0,4.759327,4.104374
12,215NV,2019-01-03,6,Baltimore,Savannah,0.0,0.0,5.383716,4.759327


In [20]:
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], format='%Y-%m-%d')

In [21]:
#'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
import re
def add_datepart(df, fldname): 
    fld = df[fldname]
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[n] = getattr(fld.dt,n.lower()).astype(int)
    #df['Elapsed'] = (fld - fld.min()).dt.days
    df.drop(fldname, axis=1, inplace=True)

In [22]:
add_datepart(df, 'FL_DATE')

In [23]:
def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):    
    df['BEGINING_OF_MONTH'] = (df['DAY_OF_MONTH'] < 10).astype('uint8')
    df['MIDDLE_OF_MONTH'] = ((df['DAY_OF_MONTH'] >= 10)&(df['DAY_OF_MONTH'] < 20)).astype('uint8')
    df['END_OF_MONTH'] = (df['DAY_OF_MONTH'] >= 20).astype('uint8')
    df['HOUR'] = df['DEP_TIME'].astype('float32').map(lambda x: x/100).astype('int32')
    df['MORNING'] = df['HOUR'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['DAY'] = df['HOUR'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['EVENING'] = df['HOUR'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['NIGHT'] = df['HOUR'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['WINTER'] = df['MONTH'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['SPRING'] = df['MONTH'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['SUMMER'] = df['MONTH'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['AUTUMN'] = df['MONTH'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['WEEKEND'] = (df['DAY_OF_WEEK'] >= 5).astype(int) 
    df['WEEKDAY'] = (df['DAY_OF_WEEK'] < 5).astype(int)
    df['DEPTIME_COS'] = df['DEP_TIME_norm'].map(make_harmonic_features_cos)
    df['DEPTIME_SIN'] = df['DEP_TIME_norm'].map(make_harmonic_features_sin)
    return df

In [24]:
df = feature_eng(df)

In [25]:
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_CITY_MARKET_ID,ORIGIN_CITY_NAME,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_CITY_MARKET_ID,DEST_CITY_NAME,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,CRS_ARR_TIME,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,AIR_TIME,DISTANCE,DIV_AIRPORT_LANDINGS,CRS_DEP_TIME_MNT,CRS_DEP_TIME_MNT_norm,DEP_TIME_MNT,DEP_TIME_norm,FLIGHT_COUNT,PRIOR_ORIGIN_CITY_NAME,PRIOR_DELAY,PRIOR_DEP_MNT,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,BEGINING_OF_MONTH,MIDDLE_OF_MONTH,END_OF_MONTH,HOUR,MORNING,DAY,EVENING,NIGHT,WINTER,SPRING,SUMMER,AUTUMN,WEEKEND,WEEKDAY,DEPTIME_COS,DEPTIME_SIN
0,2019,1,1,1,2,G4,215NV,1670,34082,Punta Gorda,12,Florida,33,34512,Rockford,17,Illinois,41,0810,0805,-5.0,0.0,0.0,-1.0,956,0.0,0.0,166.0,146.0,1127.0,0,490,2.139514,485,2.116212,1,None,0.0,0.000000,0,1,0,1,0,1,1,0,0,8,1,0,0,0,1,0,0,0,0,1,0.999985,0.005540
1,2019,1,1,1,2,G4,215NV,1671,34512,Rockford,17,Illinois,41,34082,Punta Gorda,12,Florida,33,1041,1127,46.0,46.0,1.0,3.0,1422,0.0,0.0,161.0,153.0,1127.0,0,641,2.798834,687,2.997603,2,Punta Gorda,0.0,2.139514,0,1,0,1,0,1,1,0,0,11,1,0,0,0,1,0,0,0,0,1,0.999969,0.007848
2,2019,1,1,2,3,G4,215NV,890,33195,St. Petersburg,12,Florida,33,31823,Fort Wayne,18,Indiana,42,0630,0622,-8.0,0.0,0.0,-1.0,852,0.0,0.0,142.0,124.0,912.0,0,390,1.702879,382,1.666789,1,None,0.0,0.000000,0,0,0,0,0,0,1,0,0,6,0,0,0,1,1,0,0,0,0,1,0.999990,0.004364
3,2019,1,1,2,3,G4,215NV,891,31823,Fort Wayne,18,Indiana,42,33195,St. Petersburg,12,Florida,33,0937,0929,-8.0,0.0,0.0,-1.0,1155,0.0,0.0,138.0,126.0,912.0,0,577,2.519387,569,2.482731,2,St. Petersburg,0.0,1.702879,0,0,0,0,0,0,1,0,0,9,1,0,0,0,1,0,0,0,0,1,0.999979,0.006500
4,2019,1,1,2,3,G4,215NV,1203,33195,St. Petersburg,12,Florida,33,30431,Asheville,37,North Carolina,36,1314,1308,-6.0,0.0,0.0,-1.0,1448,0.0,0.0,94.0,73.0,519.0,0,794,3.466886,788,3.438299,3,Fort Wayne,0.0,2.519387,0,0,0,0,0,0,1,0,0,13,0,1,0,0,1,0,0,0,0,1,0.999959,0.009001


#### *Categorical Features*

In [26]:
df.select_dtypes(include=['object']).columns

Index(['OP_UNIQUE_CARRIER', 'TAIL_NUM', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_NM',
       'DEST_CITY_NAME', 'DEST_STATE_NM', 'CRS_DEP_TIME', 'DEP_TIME',
       'PRIOR_ORIGIN_CITY_NAME'],
      dtype='object')

In [27]:
def label_encoder(col):
    col = LabelEncoder().fit_transform(col.astype(str))
    return col

for column in ['TAIL_NUM', 'OP_UNIQUE_CARRIER', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_NM',
               'DEST_CITY_NAME', 'DEST_STATE_NM', 'PRIOR_ORIGIN_CITY_NAME']:
    df[column] = label_encoder(df[column])

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7268231 entries, 0 to 7268230
Data columns (total 61 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   YEAR                    int64  
 1   QUARTER                 int64  
 2   MONTH                   int64  
 3   DAY_OF_MONTH            int64  
 4   DAY_OF_WEEK             int64  
 5   OP_UNIQUE_CARRIER       int32  
 6   TAIL_NUM                int32  
 7   OP_CARRIER_FL_NUM       int64  
 8   ORIGIN_CITY_MARKET_ID   int64  
 9   ORIGIN_CITY_NAME        int32  
 10  ORIGIN_STATE_FIPS       int64  
 11  ORIGIN_STATE_NM         int32  
 12  ORIGIN_WAC              int64  
 13  DEST_CITY_MARKET_ID     int64  
 14  DEST_CITY_NAME          int32  
 15  DEST_STATE_FIPS         int64  
 16  DEST_STATE_NM           int32  
 17  DEST_WAC                int64  
 18  CRS_DEP_TIME            object 
 19  DEP_TIME                object 
 20  DEP_DELAY               float64
 21  DEP_DELAY_NEW           float64

In [29]:
df.shape

(7268231, 61)

In [30]:
df.to_csv('Pre_Processed.csv', index = False)